# Serializing GMT Files to CSV

In this notebook we'll convert GMT files from [Enrichr](https://maayanlab.cloud/Enrichr) to CSV files that can be ingested in the graph database. This includes several steps:
1. Mapping/ generating ids to the terms
2. Mapping genes to their Entrez ID
3. Creating the CSV file

## CSV Serialization
Nodes and edges are serialized differently for our knowledge graph. 

### Node Serialization
Serialized nodes requires two columns: id (which can be any unique identifier like UUID, ontology id, or other persistent identifiers) and label. Optionally, you can add more columns for additional metadata. CSV file should be formatted this way: `<node_type>.node.csv` for it to be compatible with the provided ingestion script. This means for our GMT files, we need two node files: (1) label type, and (2) genes

|      |   id          |   label                                                                    |   ontology_label                                              |   uri                                                  |
|------|---------------|----------------------------------------------------------------------------|---------------------------------------------------------------|--------------------------------------------------------|
|   0  |   GO:0051084  |   'de novo' posttranslational protein folding (GO:0051084)                 |   'de novo' posttranslational protein folding                 |   http://amigo.geneontology.org/amigo/term/GO:0051084  |
|   1  |   GO:0006103  |   2-oxoglutarate metabolic process (GO:0006103)                            |   2-oxoglutarate metabolic process                            |   http://amigo.geneontology.org/amigo/term/GO:0006103  |
|   2  |   GO:0050428  |   3'-phosphoadenosine 5'-phosphosulfate biosynthetic process (GO:0050428)  |   3'-phosphoadenosine 5'-phosphosulfate biosynthetic process  |   http://amigo.geneontology.org/amigo/term/GO:0050428  |
|   3  |   GO:0050427  |   3'-phosphoadenosine 5'-phosphosulfate metabolic process (GO:0050427)     |   3'-phosphoadenosine 5'-phosphosulfate metabolic process     |   http://amigo.geneontology.org/amigo/term/GO:0050427  |
|   4  |   GO:0061158  |   3'-UTR-mediated mRNA destabilization (GO:0061158)                        |   3'-UTR-mediated mRNA destabilization                        |   http://amigo.geneontology.org/amigo/term/GO:0061158  |
|   5  |   GO:0070935  |   3'-UTR-mediated mRNA stabilization (GO:0070935)                          |   3'-UTR-mediated mRNA stabilization                          |   http://amigo.geneontology.org/amigo/term/GO:0070935  |

### Edge Serialization

Meanwhile, edges require (1) source id, (2) the relation, and (3) target id columns. The rest are optional metadata. CSV file should be formatted as follows: `<source_node_type>.<relation>.<target_node_type>.edges.csv`.

|      |   source  |   relation  |   target      |   source_label  |   target_label                                              |   resource       |   link_to_resource          |
|------|-----------|-------------|---------------|-----------------|-------------------------------------------------------------|------------------|-----------------------------|
|   0  |   23753   |   GO BP     |   GO:0051084  |   SDF2L1        |   'de novo' posttranslational protein folding (GO:0051084)  |   Gene Ontology  |   http://geneontology.org/  |
|   1  |   3313    |   GO BP     |   GO:0051084  |   HSPA9         |   'de novo' posttranslational protein folding (GO:0051084)  |   Gene Ontology  |   http://geneontology.org/  |
|   2  |   10576   |   GO BP     |   GO:0051084  |   CCT2          |   'de novo' posttranslational protein folding (GO:0051084)  |   Gene Ontology  |   http://geneontology.org/  |
|   3  |   6767    |   GO BP     |   GO:0051084  |   ST13          |   'de novo' posttranslational protein folding (GO:0051084)  |   Gene Ontology  |   http://geneontology.org/  |
|   4  |   3310    |   GO BP     |   GO:0051084  |   HSPA6         |   'de novo' posttranslational protein folding (GO:0051084)  |   Gene Ontology  |   http://geneontology.org/  |
|   5  |   957     |   GO BP     |   GO:0051084  |   ENTPD5        |   'de novo' posttranslational protein folding (GO:0051084)  |   Gene Ontology  |   http://geneontology.org/  |

In [54]:
import requests
import json
import re
from tqdm import tqdm
import os
import pandas as pd
import time
import uuid

### Gene Mapper
To start our conversion, we need a way to map the gene names to their respective gene ids. The following code downloads the metadata for Homo sapiens genes from NCBI gene and creates a mapper that returns the gene id. It does this by (1) mapping gene labels to ID, (2) mapping synonyms to ID, (3) mapping upper case gene labels and synonyms to ids. (3) is done to address the fact that Enrichr gene names are all upper case. Ambiguous labels (i.e. names with multiple ids) are removed from the map. The function `get_gene_meta` extends this and returns a dictionary containing the gene id, label, and uri which can be used for our serialization.

In [55]:
def fetch_save_read(url, file, reader=pd.read_csv, sep='\t', **kwargs):
  ''' Download file from {url}, save it to {file}, and subsequently read it with {reader} using pandas options on {**kwargs}.
  '''
  if not os.path.exists(file):
    if os.path.dirname(file):
      os.makedirs(os.path.dirname(file), exist_ok=True)
    df = reader(url, sep=sep, index_col=None)
    df.to_csv(file, sep=sep, index=False)
  return pd.read_csv(file, sep=sep, **kwargs)

In [56]:
organism = "Mammalia/Homo_sapiens"
url = 'ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/{}.gene_info.gz'.format(organism)
file = '{}.gene_info.tsv'.format(organism)

ncbi_gene = fetch_save_read(url, file)


In [57]:
def maybe_split(record):
    ''' NCBI Stores Nulls as '-' and lists '|' delimited
    '''
    if record in {'', '-'}:
        return set()
    return set(record.split('|'))

def supplement_dbXref_prefix_omitted(ids):
    ''' NCBI Stores external IDS with Foreign:ID while most datasets just use the ID
    '''
    for id in ids:
        # add original id
        yield id
        # also add id *without* prefix
        if ':' in id:
            yield id.split(':', maxsplit=1)[1]

In [58]:
ncbi_gene['All_synonyms'] = [
    set.union(
      maybe_split(gene_info['Symbol']),
      maybe_split(gene_info['Symbol_from_nomenclature_authority']),
      maybe_split(str(gene_info['GeneID'])),
      maybe_split(gene_info['Synonyms']),
      maybe_split(gene_info['Other_designations']),
      maybe_split(gene_info['LocusTag']),
      set(supplement_dbXref_prefix_omitted(maybe_split(gene_info['dbXrefs']))),
    )
    for _, gene_info in ncbi_gene.iterrows()
  ]

synonyms, gene_id = zip(*{
    (synonym, gene_info['GeneID'])
    for _, gene_info in ncbi_gene.iterrows()
    for synonym in gene_info['All_synonyms']
  })
ncbi_lookup_syn = pd.Series(gene_id, index=synonyms)
symbols, cap, gene_id = zip(*{
    (gene_info['Symbol'], gene_info['Symbol'].upper(), gene_info['GeneID'])
    for _, gene_info in ncbi_gene.iterrows()
  })
ncbi_lookup_sym = pd.Series(gene_id, index=symbols)
ncbi_lookup_sym_cap = pd.Series(gene_id, index=cap)

In [59]:
index_values = ncbi_lookup_syn.index.value_counts()
ambiguous = index_values[index_values > 1].index
ncbi_lookup_syn_disambiguated = ncbi_lookup_syn[(
(ncbi_lookup_syn.index == ncbi_lookup_syn) | (~ncbi_lookup_syn.index.isin(ambiguous))
)]
sym_dict = ncbi_lookup_sym.to_dict()
syn_dict_cap = ncbi_lookup_sym_cap.to_dict()
syn_dict = ncbi_lookup_syn_disambiguated.to_dict()
def gene_lookup(gene):
    gene_id = sym_dict.get(gene)
    if gene_id: return str(gene_id)
    gene_id = syn_dict_cap.get(gene)
    if gene_id: return str(gene_id)
    return str(syn_dict.get(gene))

In [60]:
gene_lookup("H4-16")

'None'

In [61]:
gene_lookup("STAT3")

'6774'

In [62]:
all_genes = {}
gene_ids = set()
def get_gene_meta(gene):
    if gene in all_genes:
        return all_genes[gene]
    else:
        gene_id = gene_lookup(gene)
        if gene_id in gene_ids:
            return None
        elif gene_id == 'None':
            return None
        elif gene_id == None:
            return None
        else:
            gene_ids.add(gene_id)
            all_genes[gene] = {
                "id": gene_id,
                "label": gene,
                "uri": "https://www.ncbi.nlm.nih.gov/gene/%s"%gene_id
            }
            return all_genes[gene]

get_gene_meta("COPB2")

{'id': '9276',
 'label': 'COPB2',
 'uri': 'https://www.ncbi.nlm.nih.gov/gene/9276'}

In [63]:
get_gene_meta('STAT3')

{'id': '6774',
 'label': 'STAT3',
 'uri': 'https://www.ncbi.nlm.nih.gov/gene/6774'}

### Downloading the GMT files from Enrichr
The following code downloads the GMT file from Enrichr. This function checks the existence of the file locally before downloading it.

In [64]:
def fetch_and_save_library(library, file):
  ''' Download file from {url}, save it to {file}, and subsequently read it with {reader} using pandas options on {**kwargs}.
  '''
  if not os.path.exists(file):
    if os.path.dirname(file):
      os.makedirs(os.path.dirname(file), exist_ok=True)
    gmt_url = "https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=%s"%library
    res = requests.get(gmt_url)
    gmt = res.text
    with open(file, 'w') as o:
        o.write(gmt)
  
  with open(file) as o:
    return o.read().strip().split("\n")

### Serializing GMT Files
Now that we have everything we need, let's try converting some Enrichr libraries to CSV files. Before we start let's define first a dictionary to store all the gene metadata. We'll use this to generate a combined gene node csv file later.

####  GO_Biological_Process_2021
The following code block downloads the gmt file if it does not exist

In [12]:
library = "GO_Biological_Process_2021"
filename = "gmt/%s.gmt"%library
gmt = fetch_and_save_library(library, filename)
print(gmt[0])

'de novo' posttranslational protein folding (GO:0051084)		SDF2L1	HSPA9	CCT2	ST13	HSPA6	ENTPD5	HSPA5	PTGES3	HSPA1L	HSPA8	DNAJB13	HSPA2	DNAJB14	HSPE1	DNAJC18	GAK	DNAJC7	DNAJB12	HSPA1A	HSPA1B	ERO1A	SELENOF	HSPA14	HSPA13	DNAJB1	CHCHD4	BAG1	DNAJB5	DNAJB4	SDF2	UGGT1	


![GO](./img/graph.png)

##### Serializing the nodes

In [16]:
def gene_set_name_resolver(label):
    return {
        "id": str(uuid.uuid5(uuid.NAMESPACE_URL, label)),
        "label": label
    }

In [17]:
term = gmt[0].split("\t\t")[0]

In [18]:
gene_set_name_resolver(term)

{'id': 'b48eba74-e207-575a-8219-f56e3da7c656',
 'label': "'de novo' posttranslational protein folding (GO:0051084)"}

##### Serializing the edges

Now that we have a way to get process the term and gene nodes, let's now get to serializing edges. For a GMT file, we say that an edge exists between a gene and a term if the gene is part of that term's gene set, that is if we have the following:
```
Term 1      Gene 1  Gene 2  Gene 3
Term 2      Gene 2  Gene 4  Gene 5
```
Then we say that there is an edge between Term 1 and Gene 1, Gene 2, and Gene 3, and Term 2 and Gene 2, Gene 4, and Gene 5.

In [19]:
def iterate_gmt(gmt, term_node, relation, gene_set_name_resolver, resource=None, edge_properties={}):
    terms = []
    edges = []
    for line in tqdm(gmt):
        term, *genes = line.strip().split("\t")
        genes = genes[1:]
        term_meta = gene_set_name_resolver(term)
        if term_meta:
            term_id = term_meta["id"]
            terms.append(term_meta)
            for gene in genes:
                gene_meta = get_gene_meta(gene)
                if gene_meta:
                    if type(gene_meta) == str:
                        print(gene, gene_meta)
                    gene_id = gene_meta["id"]
                    edge = {
                        "source": term_id,
                        "relation": relation,
                        "target": gene_id,
                        "source_label": term,
                        "target_label": gene,
                        **edge_properties.get((term, gene), {})
                    }
                    if resource:
                        edge["resource"] = resource
                    edges.append(edge)
    term_df = pd.DataFrame.from_records(terms)
    cols = ["id", "label"] + [i for i in term_df.columns if not i in ["id", "label"]]
    term_df = term_df[cols]
    term_df.to_csv("csv/%s.nodes.csv"%term_node, index=False)
    edge_df = pd.DataFrame.from_records(edges)
    edge_df.to_csv("csv/%s.%s.Gene.edges.csv"%(term_node, relation), index=False)

In [20]:
term_node = "GO Biological Process Term"
relation = "GO BP"
resource = "http://geneontology.org/"
iterate_gmt(gmt, term_node, relation, gene_set_name_resolver, resource)

100%|██████████| 6036/6036 [00:00<00:00, 21582.65it/s]


In [21]:
node_df = pd.read_csv('./csv/GO Biological Process Term.nodes.csv')
node_df.head()

,id,label
0,b48eba74-e207-575a-8219-f56e3da7c656,'de novo' posttranslational protein folding (G...
1,35835311-021b-5ecc-8b54-dbcd8d4d066c,2-oxoglutarate metabolic process (GO:0006103)
2,0570084d-22cc-5858-8760-1fedb82f9634,3'-phosphoadenosine 5'-phosphosulfate biosynth...
3,8f7163ac-8ad3-54c9-96f1-447be2932a3d,3'-phosphoadenosine 5'-phosphosulfate metaboli...
4,92d119a4-5bf3-5c78-b4ba-7190ccef24ad,3'-UTR-mediated mRNA destabilization (GO:0061158)


In [22]:
edge_df = pd.read_csv('./csv/GO Biological Process Term.GO BP.Gene.edges.csv')
edge_df.head()

,source,relation,target,source_label,target_label,resource
0,b48eba74-e207-575a-8219-f56e3da7c656,GO BP,23753,'de novo' posttranslational protein folding (G...,SDF2L1,http://geneontology.org/
1,b48eba74-e207-575a-8219-f56e3da7c656,GO BP,3313,'de novo' posttranslational protein folding (G...,HSPA9,http://geneontology.org/
2,b48eba74-e207-575a-8219-f56e3da7c656,GO BP,10576,'de novo' posttranslational protein folding (G...,CCT2,http://geneontology.org/
3,b48eba74-e207-575a-8219-f56e3da7c656,GO BP,6767,'de novo' posttranslational protein folding (G...,ST13,http://geneontology.org/
4,b48eba74-e207-575a-8219-f56e3da7c656,GO BP,3310,'de novo' posttranslational protein folding (G...,HSPA6,http://geneontology.org/


#### Exercise: MGI_Mammalian_Phenotype_Level_4_2021
Create a `gene_set_name_resolver` function for MGI_Mammalian_Phenotype_Level_4_2021

In [23]:
library = 'MGI_Mammalian_Phenotype_Level_4_2021'
filename = "gmt/%s.gmt"%library
gmt = fetch_and_save_library(library, filename)
print(gmt[0])

abdominal situs ambiguus MP:0011250		CCDC39	DNAH5	INVS	DNAH11	DNAAF3	FOXH1	RPGRIP1L	DRC1	DNAI1	IFT27	


In [ ]:
def gene_set_name_resolver(name):
    pass

In [24]:
term_node = "Mouse Phenotype"
relation = "MP"
resource = "http://www.informatics.jax.org"
iterate_gmt(gmt, term_node, relation, gene_set_name_resolver, resource)

100%|██████████| 4601/4601 [00:00<00:00, 15360.47it/s]


## Finding Up and Drug regulated genes from drug perturbation

In [31]:
df = pd.read_csv("https://minio.dev.maayanlab.cloud/kg-demo/lincs_consensus_drugs_5000.csv", index_col=0)
df.shape

(12327, 5000)

In [33]:
df.iloc[0:5, 0:5]

,afatinib,erlotinib,neratinib,lapatinib,pazopanib
A1CF,0.001353,0.000755,0.000463,-0.000138,-0.000276
A2M,-0.002113,-0.000268,-0.003915,0.000255,0.000854
A4GALT,-0.000360,-0.000690,-0.000118,0.000192,-0.000363
A4GNT,0.000507,0.000107,0.000174,-0.000241,-0.000167
AAAS,-0.000123,0.000050,-0.000900,0.000348,-0.000008


## Getting the top 100 up-regulated genes:
1. For each drug, we sort the genes in descending order,
2. Take the top 100 positive scoring genes

In [53]:
drug = "afatinib"
sorted = df[drug].sort_values(ascending=False)
sorted

HMOX1       0.013901
PHGDH       0.009433
NUPR1       0.009212
CHAC1       0.008992
DDIT4       0.008389
              ...   
THBS1      -0.006529
ARHGAP29   -0.006552
CCN2       -0.006792
AREG       -0.007930
TACSTD2    -0.009251
Name: afatinib, Length: 12327, dtype: float64

In [37]:
## Top 100 up regulated genes for afatinib
sorted.head(100)

HMOX1        0.013901
PHGDH        0.009433
NUPR1        0.009212
CHAC1        0.008992
DDIT4        0.008389
               ...   
XBP1         0.003288
C14orf132    0.003267
PFN2         0.003262
SLC7A5       0.003246
SLC1A4       0.003242
Name: afatinib, Length: 100, dtype: float64

## Putting it all together

In [40]:
for k,v in sorted.head(100).items():
    print(k,v)

HMOX1 0.013901103
PHGDH 0.009433473
NUPR1 0.00921154
CHAC1 0.008991934
DDIT4 0.00838907
ASNS 0.00814286
ASS1 0.0070086727
CBS 0.006973538
GPNMB 0.0068709115
PCK2 0.0068163862
TSC22D3 0.006695175
PSAT1 0.006614989
SLC3A2 0.0065062684
TRIB3 0.00612757
ABHD4 0.0060702595
XPOT 0.006007641
SCD 0.0059088995
METTL7A 0.0058731255
DDIT3 0.0057463204
SLC7A11 0.0055631665
AARS 0.0055097663
INSIG1 0.005366992
SLC7A1 0.005351075
PIR 0.0052302293
GNPDA1 0.0051882947
GARS 0.005157598
RRAGD 0.005115755
FAM129A 0.0051055225
ATF4 0.005074978
CHI3L1 0.0049606557
EPHX1 0.0049292045
SQSTM1 0.0049190386
FTH1 0.0048750304
CEBPG 0.0047666086
AKR1C3 0.004728251
MTHFD2 0.004725242
SHMT2 0.0046566958
SLC7A8 0.00455205
RAB29 0.004511242
BLVRB 0.0044702897
PHYH 0.004343569
SLC7A7 0.004319546
YARS 0.0042936914
PSPH 0.0042814375
FAM102A 0.004236382
NQO1 0.004228466
AKR1B10 0.004225984
IMPA2 0.0042155804
HTATIP2 0.0041666324
INHBE 0.004164592
LY96 0.0041365405
EIF4EBP1 0.0041180495
SPINK1 0.004090231
HSPA1A 0.0040218

In [41]:
drugs = []
edges = []
for drug in df.columns:
    ## generate ids for drugs
    drug_id = uuid.uuid5(uuid.NAMESPACE_URL, drug)
    drugs.append({
        "id": drug_id,
        "label": drug
    })
    sorted = df[drug].sort_values(ascending=False)
    up_regulated = sorted.head(100)
    for gene, score in up_regulated.items():
        gene_meta = get_gene_meta(gene)
        if gene_meta:
            if type(gene_meta) == str:
                print(gene, gene_meta)
            gene_id = gene_meta["id"]
            edge = {
                "source": drug_id,
                "relation": "LINCS Up Regulated",
                "target": gene_id,
                "source_label": drug,
                "target_label": gene,
                "score": score
            }
            edges.append(edge)


In [42]:
drug_df = pd.DataFrame.from_records(drugs)
cols = ["id", "label"] + [i for i in drug_df.columns if not i in ["id", "label"]]
drug_df = drug_df[cols]
drug_df.to_csv("csv/Drug.nodes.csv", index=False)
edge_df = pd.DataFrame.from_records(edges)
edge_df.to_csv("csv/Drug.LINCS Up Regulated.Gene.edges.csv", index=False)

In [44]:
# Exercise: Write a code for the down regulated genes

## Gene.csv
Using the `all_genes` dictionary, create a `Genes.nodes.csv` file

In [45]:
# write your code here
genes = pd.DataFrame.from_records([i for i in all_genes.values() if not i == None])
genes.to_csv("csv/Gene.nodes.csv", index=False)

In [46]:
gene_df = pd.read_csv('csv/Gene.nodes.csv')
gene_df.head()

,id,label,uri
0,9276,COPB2,https://www.ncbi.nlm.nih.gov/gene/9276
1,6774,STAT3,https://www.ncbi.nlm.nih.gov/gene/6774
2,23753,SDF2L1,https://www.ncbi.nlm.nih.gov/gene/23753
3,3313,HSPA9,https://www.ncbi.nlm.nih.gov/gene/3313
4,10576,CCT2,https://www.ncbi.nlm.nih.gov/gene/10576


#### Ingestion
Ingestion is relatively simple if we followed followed the naming convention. `src/import_csv.py` is provided to do the heavy lifting. To run it just type the following on the command line:
```
python import_csv.py /path/to/csv/directory
```
This will run a bulk import of your csv files
e.g.
```
python import_csv.py ../notebooks/csv
```